# Pull images to classify with MakeSense.ai

Randomly pulled, tracked, for use with [MakeSense.ai](https://www.makesense.ai/) as a possible annotation interface.

Steps are:
1. Run the ocr_and_image_processing_batch.py file
1. Run this file

In your tmp storage (as defined in the config file) -- there will now be a MakeSense directory
 * go to makesense.ai and copy images into the upload box (in `<tmpfolder>/images`) and select Object Detection as your makesense.ai task
 * go to Actions -> Import Annotations and import labels and annotations with the documents in `<tmpfolder>/annotations` -- make sure to select "multiple files" (not a single COCO file) for the upload.
 * When finished annotating do Actions -> Export Annotations -> as *single* CSV file
 * Move this file to your `check_makesense` directory.
 

In [114]:
import config

In [115]:
# PMC PubLayNet
check_makesense = '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/Annotations_htrc/MakeSenseAnnotations/'
save_makesense = check_makesense
ocr_results_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/OCR_processing_htrc/'
images_jpeg_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/Pages_htrc/RandomSingleFromPDFIndexed/'
check_json_dir = None

# # defaults
# ocr_results_dir = None
# images_jpeg_dir = None
# check_json_dir = None

In [116]:
# Use prior weights to seed the boxes -- these come from the training
binary_dirs = 'binaries_model12_tfrecordz/'
weightsFileDir = config.save_weights_dir +'saved_weights/'+'20211218_model12tfz/'
weightsFile = 'training_1model12_tfrec_model_l0.019131713.h5'  

In [117]:
# how many you wanna grab?
nRandom = 100

# invert colors?
invert_colors = False

In [118]:
# labels for annotations:
labels = ['figure', 'figure caption', 'table', 'math formula', 'sub fig caption', 'colorbar', 'NotSure', 'no label'] # no label is for nothing on the page

In [119]:
# import post_processing_utils
# from importlib import reload
# reload(post_processing_utils)

In [120]:
import pandas as pd
from glob import glob
import numpy as np
import pickle
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import cv2 as cv

from annotation_utils import get_all_ocr_files, collect_ocr_process_results
from mega_yolo_utils import build_predict
from feature_generation_utils import generate_single_feature
from post_processing_utils import get_ocr_results,get_image_process_boxes, \
    clean_merge_heurstic_captions, clean_overlapping_squares, clean_found_overlap_with_ocr, \
    clean_merge_squares, clean_big_captions, clean_match_fig_cap, expand_found_boxes_fig_cap, \
    expand_found_area_above_cap

In [121]:
if images_jpeg_dir is None: images_jpeg_dir = config.images_jpeg_dir

In [122]:
# save labels
# save feature list
with open(save_makesense +'saved_labels.pickle', 'wb') as ff:
    pickle.dump([labels], ff)   

In [123]:
ocrFiles = get_all_ocr_files(ocr_results_dir=ocr_results_dir)
ocrFiles

['/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/OCR_processing_htrc/full_ocr_newPDFs_TIFF_take5.pickle',
 '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/OCR_processing_htrc/full_ocr_newPDFs_TIFF_take3.pickle',
 '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/OCR_processing_htrc/full_ocr_newPDFs_TIFF_take1.pickle',
 '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/OCR_processing_htrc/full_ocr_newPDFs_TIFF_take4.pickle',
 '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/OCR_processing_htrc/full_ocr_newPDFs_TIFF_take2.pickle']

In [124]:
# import annotation_utils
# reload(annotation_utils)
# from annotation_utils import collect_ocr_process_results

In [125]:
print('retreiving OCR data, this can take a moment...')
ws1, paragraphs1, squares1, html1, rotations1,colorbars1 = collect_ocr_process_results(ocrFiles)

retreiving OCR data, this can take a moment...
##### OCR retrieval FILE: on 1 of 5  ##### 
--- OCR retrieval: on 0 of 50 ---
##### OCR retrieval FILE: on 2 of 5  ##### 
--- OCR retrieval: on 0 of 20 ---
##### OCR retrieval FILE: on 3 of 5  ##### 
--- OCR retrieval: on 0 of 29 ---
##### OCR retrieval FILE: on 4 of 5  ##### 
--- OCR retrieval: on 0 of 25 ---
##### OCR retrieval FILE: on 5 of 5  ##### 
--- OCR retrieval: on 0 of 20 ---


## Generally start here for same dataset!

In [126]:
# get all dones already

# generate donesfile from list
if type(check_makesense) != list:
    lfiles = glob(check_makesense+'labels*csv')
else:
    lfiles = glob(check_makesense[0]+'labels*csv')
    for c in check_makesense[1:]:
        lfiles.extend(glob(c+'labels*csv'))

l2 = glob(save_makesense+'labels*csv')
lfiles.extend(l2)
filenames = []; labelss = []; xmins = []; ymins=[]; xmaxs=[]; ymaxs=[]; widths=[]
fnameSave = []; heights = []
for il,l in enumerate(lfiles):
    with open(l,'r') as ftrial:
        text = ftrial.readline()
        if 'label_name' in text: # have header, overwrite
            d = pd.read_csv(l, 
                            names=['label','xmin','ymin','xmax','ymax', 'fname','x','y'],
                           skiprows=1)
        else: # no header
            d = pd.read_csv(l, names=['label','xmin','ymin','xmax','ymax', 'fname','x','y'])

    #d = pd.read_csv(l, names = ['label','xmin','ymin','xmax','ymax', 'fname','x','y'])
    #d = d.drop_duplicates(subset='fname')
    for idd, dd in enumerate(d['fname'].values):
        fn = dd[:dd.rfind('.')+1]
        if fn[-1] == '.': fn = fn[:-1]
        filenames.append(fn)
        fnameSave.append(dd)
    if il == 0:
        dfTmp = d.copy()
    else:
        dfTmp = dfTmp.append(d)
# get unique
fnameSave,uind = np.unique(fnameSave,return_index=True)
# loop and grab
for idd, dd in enumerate(fnameSave):
    mask = dfTmp['fname'] == dd
    labelss.append(dfTmp.loc[mask]['label'].values); xmins.append(dfTmp.loc[mask]['xmin'].values)
    ymins.append(dfTmp.loc[mask]['ymin'].values); 
    # widths and heights!!
    xmaxs.append(dfTmp.loc[mask]['xmax'].values+dfTmp.loc[mask]['xmin'].values); 
    ymaxs.append(dfTmp.loc[mask]['ymax'].values+dfTmp.loc[mask]['ymin'].values)
    widths.append(dfTmp.loc[mask]['x'].values[0]); heights.append(dfTmp.loc[mask]['y'].values[0])
dones = pd.DataFrame({'filename':np.array(filenames)[uind], 'labels':labelss, 'xmin':xmins, 
                      'ymin':ymins, 'xmax':xmaxs,'ymax':ymaxs, 'width':widths, 'height':heights})

/var/folders/t7/bwcvd_4177q4872gxghn7p9r0000gq/T/ipykernel_29062/2011718989.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfTmp = dfTmp.append(d)
/var/folders/t7/bwcvd_4177q4872gxghn7p9r0000gq/T/ipykernel_29062/2011718989.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfTmp = dfTmp.append(d)
/var/folders/t7/bwcvd_4177q4872gxghn7p9r0000gq/T/ipykernel_29062/2011718989.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfTmp = dfTmp.append(d)
/var/folders/t7/bwcvd_4177q4872gxghn7p9r0000gq/T/ipykernel_29062/2011718989.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfTmp = dfTmp.append(d)
/var/folders/t7/bwcvd_4177q4872g

In [127]:
dones

,filename,labels,xmin,ymin,xmax,ymax,width,height
0,2003_uc1-31210014680225-1666394091_p1,"[no_label, no_label]","[2766, 2766]","[3223, 3223]","[2924, 2924]","[3407, 3407]",3161,3683
1,2003_uc1-31210014680225-1666394091_p102,"[table, table]","[256, 256]","[240, 240]","[2354, 2354]","[3278, 3278]",2652,3504
2,2003_uc1-31210014680225-1666394091_p118,"[table, table]","[266, 266]","[154, 154]","[2157, 2157]","[3192, 3192]",2727,3487
3,2003_uc1-31210014680225-1666394091_p127,"[table, table]","[349, 349]","[198, 198]","[2232, 2232]","[3298, 3298]",2719,3525
4,2003_uc1-31210014680225-1666394091_p136,"[table, table]","[243, 243]","[192, 192]","[1911, 1911]","[3228, 3228]",2619,3491
...,...,...,...,...,...,...,...,...
80,uiug-30112101602172-1666102417_p3,"[table, table]","[487, 487]","[9, 9]","[2326, 2326]","[3052, 3052]",2372,3262
81,uiug-30112101602172-1666102417_p4,"[no_label, no_label]","[2080, 2080]","[2851, 2851]","[2199, 2199]","[3014, 3014]",2377,3258
82,uiug-30112101602172-1666102417_p5,"[no_label, no_label]","[2106, 2106]","[2963, 2963]","[2286, 2286]","[3143, 3143]",2372,3258
83,uiug-30112101602172-1666102417_p6,"[no_label, no_label]","[2080, 2080]","[2854, 2854]","[2199, 2199]","[3017, 3017]",2377,3262


In [128]:
donefiles = dones['filename'].values
len(donefiles), len(np.unique(donefiles))
# debug
#donefiles = ['ofx163.1581.PMC5630905_p1.jpeg']

(85, 85)

In [129]:
# count how many done per class
labels_done_count = np.zeros(len(labels))
for i in range(len(dones)):
    for l in dones.iloc[i]['labels']:
        indd = labels.index(l.replace('_',' '))
        labels_done_count[indd] += 1
print('Already classified:')
for i in range(len(labels)):
    print(labels[i], labels_done_count[i])

Already classified:
figure 36.0
figure caption 38.0
table 56.0
math formula 54.0
sub fig caption 0.0
colorbar 0.0
NotSure 0.0
no label 84.0


In [130]:
len(donefiles)

85

In [131]:
# also, get list of error'd out scanbanks
if check_json_dir is not None:
    sbcheck = []
    for i in range(len(donefiles)):
        sbcheck.append(donefiles[i].split('_p')[0])
    #sbcheck = np.unique(sbcheck).tolist()

    # check for json file
    #no_json = []
    #for f in sbcheck:
    #    f+'/'+pdfbase+'deepfigures-results.json'
    donefiles = donefiles.tolist()
    for f in sbcheck:
        fc = check_json_dir + f
        ff = glob(fc+'/*')[0]
        if not os.path.isfile(ff + '/' + f + 'deepfigures-results.json'):
            donefiles.append(f)
            #print('yes')
    donefiles = np.unique(donefiles)

In [132]:
len(donefiles)

85

In [133]:
# create dataframe
ws, paragraphs, squares, html, rotations,colorbars = [],[],[],[],[],[]
for w,p,s,h,r,c in zip(ws1,paragraphs1, squares1, html1, rotations1,colorbars1):
    if w.split('.jpeg')[0] not in donefiles:
        ws.append(w); paragraphs.append(p);squares.append(s)
        html.append(h); rotations.append(r); colorbars.append(c)
df = pd.DataFrame({'ws':ws, 'paragraphs':paragraphs, 'squares':squares, 
                   'hocr':html, 'rotation':rotations, 'colorbars':colorbars})#, 'pdfwords':pdfwords})
df = df.drop_duplicates(subset='ws')
df = df.set_index('ws')

In [134]:
print('you have', len(df), 'pages that you can pull from')

you have 50 pages that you can pull from


In [135]:
if nRandom < len(ws):
    # grab randomly
    ind = np.random.choice(range(len(df)),nRandom,replace=False)
    #ws = np.array(ws)[ind]
else:
    #ws = np.array(ws)
    ind = np.arange(0,len(df))

In [136]:
dfout = df.iloc[ind]

In [137]:
dfout.head()

,paragraphs,squares,hocr,rotation,colorbars
ws,,,,,
osu-32435023323769-1666273876_p164.jpeg,"[(485, 186, 1053, 41), (485, 186, 1053, 41), (...","[[[453, 303], [1962, 303], [1962, 1761], [453,...","<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[]
1989_uc1-32106020677263-1666572876_p96.jpeg,"[(359, 263, 1814, 44), (359, 263, 1814, 44), (...",[],"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[]
osu-32435023323769-1666273876_p148.jpeg,"[(365, 246, 59, 31), (1178, 252, 339, 31), (11...","[[[884, 2000], [1871, 2000], [1871, 2739], [88...","<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[1655, 2078], [1655, 2167], [1667, 2167], [1..."
1989_uc1-32106020677263-1666572876_p137.jpeg,"[(286, 273, 1886, 45), (286, 273, 1886, 45), (...",[],"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[]
1989_uc1-32106020677263-1666572876_p78.jpeg,"[(2199, 377, 3, 22), (2197, 1417, 4, 19), (217...","[[[638, 1653], [1998, 1653], [1998, 3014], [63...","<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[]


In [138]:
# ws = dfout.index
# ws
#dfout.index.values.astype('str')

In [139]:
# save label file for annotations
storeTmps = config.tmp_storage_dir + 'MakeSense/'
if not os.path.exists(storeTmps):
    os.mkdir(storeTmps)
# remove, remake
shutil.rmtree(storeTmps)
os.mkdir(storeTmps)

Copy over random images:

In [140]:
# start on empty
if not os.path.exists(storeTmps+'images/'):
    os.makedirs(storeTmps+'images/')
# delete and remake
shutil.rmtree(storeTmps+'images/')
os.makedirs(storeTmps+'images/')

In [141]:
# move files
imgSizes = []; imgSizesOrig = []
for iw,w in enumerate(dfout.index.values.astype('str')):
    #print(w)
    w = images_jpeg_dir + w
    if not invert_colors:
        shutil.copyfile(w, storeTmps+'images/'+ w.split('/')[-1])
        imgSizes.append(Image.open(w).convert('RGB').size)
    else: # invert B/W
        print('not implemented!!')
        import sys; sys.exit()

In [142]:
# write label file
with open(storeTmps + 'labels.txt','w') as f:
    for l in labels:
        f.write(l.replace(' ', '_') + '\n')

For annotations, try to predict with our model:

In [143]:
# get model to run on pages
# read in anchors
saveFileAnchors = weightsFileDir + 'anchors.pickle'
with open(saveFileAnchors, 'rb') as f:
    anchors = pickle.load(f) 
    anchors = anchors.astype('float32')

    
feature_dir = config.save_binary_dir + binary_dirs
# how many features
with open(feature_dir +'feature_list.pickle', 'rb') as ff:
    feature_list = pickle.load(ff)[0]
# HACK
#feature_list = ['grayscale','fontsize','x_ascenders','x_decenders', 'word confidences', 
#                'fraction of numbers in a word','fraction of letters in a word','punctuation', 
#                'text angles','Spacy POS']

n_features = len(feature_list)

# labels file for originally trained
LABELS=pd.read_csv(feature_dir + 'LABELS.csv',names=['labels'])['labels'].values.astype('str')
CLASS = len(LABELS)

# build the model
weightsFileDownload = weightsFileDir + weightsFile
anchorsFile = weightsFileDir + 'anchors.pickle'  # should this be changed....

model = build_predict(weightsFileDownload, anchorsFile, 
                    feature_dir,LABELS,version=config.version, 
                      debug=False,n_features=n_features)
model.load_weights(weightsFileDownload)

In [144]:
#imgSizes

In [145]:
# reload(post_processing_utils)
# from post_processing_utils import get_ocr_results,get_image_process_boxes, \
#     clean_merge_heurstic_captions, clean_overlapping_squares, clean_found_overlap_with_ocr, \
#     clean_merge_squares, clean_big_captions, clean_match_fig_cap, expand_found_boxes_fig_cap, \
#     expand_found_area_above_cap

In [146]:
import tensorflow as tf

# # for tfrecrords, get datasets
# test_list = glob(feature_dir + 'test_*tfrecords')

# #if not use_training:
# test_raw_data = tf.data.TFRecordDataset(filenames=test_list, 
#                                          compression_type='GZIP', 
#                                          buffer_size=None, 
#                                         num_parallel_reads=tf.data.AUTOTUNE)
# Create a dictionary describing the features.
image_feature_description = {
    'nbox': tf.io.FixedLenFeature([], tf.float32),
    'nfeatures': tf.io.FixedLenFeature([], tf.float32),
    'boxes': tf.io.FixedLenFeature([], tf.string),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'image_name': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function_test(example_proto,anchors,CLASS):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    # parse the data
    nboxes = image_features['nbox']
    nfeatures = image_features['nfeatures']
    images_raw = image_features['image_raw']
    image = tf.io.decode_raw(images_raw,tf.float32)
    image = tf.reshape(image,[config.IMAGE_H,config.IMAGE_W,nfeatures])
    img_name = tf.cast(image_features['image_name'],tf.string)
    return image,img_name

# test_dataset = test_raw_data.map(lambda example_proto:_parse_image_function_test(example_proto,
#                                                                              anchors,CLASS))


def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
 
# Create a dictionary with features that may be relevant.
def image_example(image, boxes, img_name):
    #image_shape = tf.io.decode_jpeg(image_string).shape
    image_string = image.astype('float32')/255.0
    image_string = image.reshape(image.shape[0]*image.shape[1]*image.shape[2])

    nfeatures = image.shape[2]
    nboxes = boxes.shape[0]
    if nboxes>0:
        boxout = boxes.reshape(boxes.shape[0]*boxes.shape[1])
    else:
        boxout = np.array([])

    feature = {
      'nbox': _float_feature(np.float32(nboxes)),
      'nfeatures': _float_feature(np.float32(nfeatures)),
      'boxes': _bytes_feature(boxout.astype('float32').tobytes()),
      'image_raw': _bytes_feature(image_string.astype('float32').tobytes()),
      'image_name': _bytes_feature(img_name.encode('utf-8')),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))      

compress = 'GZIP'
tf_record_options = tf.io.TFRecordOptions(compression_type = compress) 


In [147]:
# make file into tf record

In [ ]:
if not os.path.exists(storeTmps+'annotations/'):
    os.makedirs(storeTmps+'annotations/')
# delete and remake
shutil.rmtree(storeTmps+'annotations/')
os.makedirs(storeTmps+'annotations/')

# save tmp binaries for this
if not os.path.exists(storeTmps+'binaries/'):
    os.makedirs(storeTmps+'binaries/')
# delete and remake
shutil.rmtree(storeTmps+'binaries/')
os.makedirs(storeTmps+'binaries/')

# just for counting
maxboxes = 50 # I think this is actually a place holder...

boxesout,labelsout = [],[]

#for i in range(len(dfout)):
#i = 6 #4
#i=0
#if True:
#icount = 0
#for image,images_name in test_dataset:
    #imgs_name = images_name.numpy().decode('utf-8')
for i in range(len(dfout)):
    
    if i%25 == 0: print('on', i+1, 'of', len(dfout))

    dfsingle = dfout.iloc[i]
    # if we've made it this far, let's generate features
    img_name, font = generate_single_feature(dfsingle, LABELS, maxboxes, 
                                           feature_list = feature_list, 
                                           binary_dir = storeTmps+'binaries/',
                                                images_jpeg_dir = images_jpeg_dir,
                                                astype='npz', 
                                                 npzcompressed=True) 
    
    image_np = np.load(img_name)['arr_0']
    fakebox = np.random.random([20,5])
    tf_example = image_example(image_np,fakebox,img_name)
    # write temp
    record_file = storeTmps+'binaries/tmpfile'
    with tf.io.TFRecordWriter(record_file, options=tf_record_options) as writer:
        writer.write(tf_example.SerializeToString())
        
    test_raw_data = tf.data.TFRecordDataset(filenames=record_file, 
                                             compression_type='GZIP', 
                                             buffer_size=None, 
                                            num_parallel_reads=tf.data.AUTOTUNE)
    test_dataset = test_raw_data.map(lambda example_proto:_parse_image_function_test(example_proto,
                                                                                 anchors,CLASS))

    for image,images_name in test_dataset:
        imgs_name = images_name.numpy().decode('utf-8')
    
        # predict squares in 2 ways
        # 1. MEGA YOLO
        #image_np = np.load(img_name)['arr_0']
        #image_np = image_np.astype(np.float32) / 255.0 
        # backtorgb,image_np,rotatedImage,rotatedAngleOCR,bbox_hocr,\
        #   bboxes_words,bbsq,cbsq, rotation,bbox_par = get_ocr_results(img_name, None,dfout.iloc[[i]],
        #                                                              image_np=image_np,
        #                                                              width=imgSizes[i][0], 
        #                                                              height=imgSizes[i][1],
        #                                                              images_jpeg_dir=images_jpeg_dir)
        backtorgb,image_np,rotatedImage,rotatedAngleOCR,bbox_hocr,\
          bboxes_words,bbsq,cbsq, rotation,bbox_par = get_ocr_results(img_name, None,dfout.iloc[[i]],
                                                                     image_np=image.numpy(),
                                                                     width=imgSizes[i][0], 
                                                                     height=imgSizes[i][1],
                                                                     images_jpeg_dir=images_jpeg_dir)

        boxesh, scoresh, labelsh = model.predict(image_np[np.newaxis, ...])
        #import sys; sys.exit()
        boxes1, scores1, labels1 = np.squeeze(boxesh, 0), \
          np.squeeze(scoresh, 0), np.squeeze(labelsh, 0)
        # only non -1 ones
        boxes1 = boxes1[labels1>-1]
        scores1 = scores1[labels1>-1]
        labels1 = labels1[labels1>-1]    

        # # post process the thing
        # # get OCR results and parse them, open image for image processing
        # image = 


        # get figures and captions from image processing
        captionText_figcap, bbox_figcap_pars = get_image_process_boxes(backtorgb, 
                                                                       bbox_hocr, 
                                                                       rotatedImage)
        # clean overlapping squares
        # if squares are majorly overlapping, take the one with the highest score
        sboxes_cleaned, slabels_cleaned, sscores_cleaned = clean_overlapping_squares(boxes1,
                                                                                     scores1,
                                                                                     labels1,
                                                                                     img_name)
        # combine figure caption boxes with heuristically found ones
        # -- often the heurstically found boxes are more accurate, especially 
        # in the vertical direction
        boxes_heur, labels_heur, scores_heur,\
          ibbOverlap, boxes_heur_tf = clean_merge_heurstic_captions(sboxes_cleaned, 
                                                     slabels_cleaned, 
                                                     sscores_cleaned, 
                                                bbox_figcap_pars, LABELS,None,
                                                    width=imgSizes[i][0], 
                                                    height=imgSizes[i][1])
        # NO PDF

        # other way -- w/o adding more heursitic caps:
        boxes_par_found, labels_par_found, \
          scores_par_found = clean_found_overlap_with_ocr(boxes_heur, labels_heur, 
                                                    scores_heur,bboxes_words,
                                                          bbox_par,rotation,
                                                          LABELS, None, boxes_heur_tf,
                                                          width=imgSizes[i][0], 
                                                    height=imgSizes[i][1])

        # if figure boxes are smaller than image-processing found boxes, merge them; 
        # also, do with colorbars as well if requested
        boxes_sq1, labels_sq1, scores_sq1, bbsq = clean_merge_squares(bbsq, cbsq,
                                                                boxes_par_found, 
                                                                labels_par_found, 
                                                                scores_par_found, 
                                                                LABELS, None, 
                                                                      width=imgSizes[i][0], 
                                                                      height=imgSizes[i][1])

        # if there are any huge captions -- like 75% of the area of the page or more
        #. these are wrong, so drop them
        boxes_sq2, labels_sq2, scores_sq2 = clean_big_captions(boxes_sq1,
                                                            labels_sq1,
                                                            scores_sq1, 
                                                            LABELS)
            
        # sometimes captions are slightly overlapping with figures -- split the 
        # difference between those where they touch on the "bottom"
        # Default to captions found with mega yolo, if there is a figure but 
        #. no caption found, then see if there is a heuristically found caption
        boxes_sq3, labels_sq3, scores_sq3 = clean_match_fig_cap(boxes_sq2,
                                                                 labels_sq2,
                                                             scores_sq2, bbsq,
                                                             LABELS.tolist(), 
                                                             rotatedImage, 
                                                             rotatedAngleOCR,
                                                             None, 
                                                            width=imgSizes[i][0], 
                                                            height=imgSizes[i][1])
        
        # again for found boxes?  I feel like maybe not the one above?
        boxes_sq4, labels_sq4, scores_sq4 = expand_found_boxes_fig_cap(boxes_sq3, 
                                                                    labels_sq3, 
                                                                    scores_sq3,
                                                                       bbsq,
                                                                    rotatedImage, 
                                                                    LABELS.tolist(), None, 
                                                            width=imgSizes[i][0], 
                                                            height=imgSizes[i][1])
        
        # same for found
        boxes_sq5, labels_sq5, scores_sq5 = expand_found_area_above_cap(boxes_sq4, 
                                                                        labels_sq4, 
                                                                        scores_sq4, 
                                                                        bbsq,
                                                                        rotatedImage, 
                                                                        LABELS.tolist(), None, 
                                                            width=imgSizes[i][0], 
                                                            height=imgSizes[i][1])
        
        # save it!
        #boxesout.append(boxes_sq5); labelsout.append(labels_sq5)
        
    fn = imgs_name
        
    # write this out!
    if os.path.exists(fn): 
        with open(storeTmps+'annotations/'+ fn.split('/')[-1].split('.npz')[0] + '.txt','w') as fsave:
            for fb,ll in zip(boxes_sq5,labels_sq5):
                xmin = fb[0]/config.IMAGE_W#*img.shape[1]
                xmax = fb[2]/config.IMAGE_W#*img.shape[1]
                ymin = fb[1]/config.IMAGE_H#*img.shape[0]
                ymax = fb[3]/config.IMAGE_H#*img.shape[0]

                #xmin = bb[0]*xfrac; ymin = bb[1]*yfrac; xmax = bb[2]*xfrac; ymax = bb[3]*yfrac
                #x = xmin/imSize[1]; y = ymin/imSize[0]; 
                #w = (xmax-xmin)/imSize[1]; h = (ymax-ymin)/imSize[0]
                x = xmin; y = ymin
                w = xmax-xmin; h = ymax-ymin
                # I think we want centers?
                x = x+w*0.5; y = y+0.5*h
                if ll > -1:
                    lab = LABELS[int(ll)]
                    if lab == 'multi-figure': lab = 'figure' # rename all to figure for now
                    try: 
                        l = labels.index(lab)
                    except:
                        l = -1
                    if l > -1: # found in these labels
                        if w<0: w=0.5
                        if h<0: h=0.5
                        fsave.write(str(l) + ' ' + str(max([x,0])) + ' ' + str(max([y,0])) + ' ' + str(min([w,1])) + ' ' + str(min([h,1])) + '\n')
                        
            # if no boxes -- no label labels
            if len(boxes_sq5) == 0:
                x = 0.90; y = 0.90; w = 0.05; h=0.05
                fsave.write(str(labels.index('no label')) + ' ' + str(max([x,0])) + ' ' + str(max([y,0])) + ' ' + str(min([w,1])) + ' ' + str(min([h,1])) + '\n')

    else:
        print('something has gone wrong')
        import sys; sys.exit()


#write label file with ALL labels
with open(storeTmps + 'annotations/'  + 'labels.txt','w') as f:
    for l in labels:
        f.write(l.replace(' ', '_') + '\n')

print('all done!  go classify!')

on 1 of 50
1/1 [==============================] - 0s 483ms/step
1989_uc1-32106020677263-1666572876_p221.jpeg something has happened with rotation -- page says angle = 180 words say angle = 0.0
